datos

In [7]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.7/633.7 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 29.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.4.0 which is incompatible.
2.18.0-dev20240727


In [8]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-07-27 18:14:24--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.01s   

2024-07-27 18:14:24 (26.9 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-07-27 18:14:24--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd


"""Para leer archivos TSV (Tab-Separated Values) en pandas, podemos usar la función pd.read_csv()
 con el parámetro sep especificado como '\t' para indicar que los valores están separados por tabulaciones"""
column_names = ["label", "text"]
df_train =  pd.read_csv(train_file_path, sep = "\t", names = column_names)
df_test = pd.read_csv(test_file_path, sep= "\t", names = column_names)


In [12]:
# evaluamos cantidad de columnas
print(df_test.head().shape)

# En la salida vemos si
df_test

(5, 2)


,label,text
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


# modelo

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [17]:

# Supongamos que df_train y df_test son tus DataFrames
df = pd.concat([df_train, df_test], ignore_index=True)


In [18]:
df

,label,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
5566,ham,true dear..i sat to pray evening and felt so.s...
5567,ham,"what will we do in the shower, baby?"
5568,ham,where are you ? what are you doing ? are yuou ...
5569,spam,ur cash-balance is currently 500 pounds - to m...


In [19]:
# # Ejemplo de DataFrame
# data = {'text': ['free prize', 'important meeting', 'win money', 'project deadline', 'click to win'],
#         'label': ['spam', 'ham', 'spam', 'ham', 'spam']}
# df = pd.DataFrame(data)

# Convertir las etiquetas a valores binarios
df['label'] = df['label'].map({'ham': 0, 'spam': 1})


In [20]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)


In [21]:
vectorizer = CountVectorizer() # tokenizamos
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)


In [22]:
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)


MultinomialNB()

In [23]:
y_pred = clf.predict(X_test_counts)

# Evaluar el modelo
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))


Accuracy: 0.9775784753363229
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       975
           1       0.95      0.87      0.91       140

    accuracy                           0.98      1115
   macro avg       0.96      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [38]:
def predict_message(text, vectorizer = vectorizer, model = clf):
    """
    Predice si un texto es 'spam' o 'ham' usando un modelo entrenado.

    Args:
    - text (str): El texto a clasificar.
    - vectorizer (CountVectorizer): El vectorizador usado para transformar el texto.
    - model (MultinomialNB): El modelo entrenado.

    Returns:
    - str: 'spam' o 'ham'.
    """
    # Transformar el texto usando el vectorizador
    text_counts = vectorizer.transform([text])

    # Hacer la predicción
    prediction = model.predict(text_counts)[0]

    # Mapear la predicción a 'spam' o 'ham'
    return 'spam' if prediction == 1 else 'ham'

# Ejemplo de uso
# example_text = "Congratulations, you've won a prize!" # bien  spam
# example_text = "our new mobile video service is live. just install on your phone to start watching."  # bien spam
# example_text = "how are you doing today" # ham # good
example_text = "sale today! to stop texts call 98912460324" # excelente # spam
print(predict_message(example_text, vectorizer, clf))

# works very good!


spam


In [37]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
